In [ ]:
%pip install ../dist/alquimia_fair_forge-0.0.1.tar.gz -q

In [ ]:
from fair_forge.metrics import Context
from pydantic import SecretStr
from helpers.retriever import LocalRetriever
from getpass import getpass
from elasticsearch import Elasticsearch, helpers
import os

In [3]:
JUDGE_API_KEY = SecretStr(os.environ.get("JUDGE_API_KEY"))
ELASTIC_URL = os.environ.get('ELASTIC_URL')
ELASTIC_AUTH = [os.environ.get('ELASTIC_AUTH_USER'), os.environ.get('ELASTIC_AUTH_PASSWORD')]
RUN_NAME= os.environ.get("run_name")
index_name = f"{RUN_NAME}-bias"

In [ ]:
metrics = Context.run(
    LocalRetriever,
    judge_api_key=JUDGE_API_KEY
)

In [ ]:
es = Elasticsearch(
    ELASTIC_URL,
    basic_auth=tuple(ELASTIC_AUTH),
)

In [ ]:
def recreate_index(index_name: str, mapping: dict):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created.")

In [ ]:
mapping_contextualizer = {
  "mappings": {
    "properties": {
      "session_id": {"type": "keyword"},
      "context": {"type": "text"},
      "context_insight": {"type": "text"},
      "context_awareness": {"type": "float"},
      "context_thinkings": {"type": "text"},
      "qa_id": {"type": "keyword"},
      "assistant_id": {"type": "keyword"},
    }
  }
}

recreate_index(index_name, mapping_contextualizer)

In [ ]:
docs = []
for metric in metrics:
    docs.append({
            "_index": index_name,
            "_source": metric.model_dump()
    })

helpers.bulk(es, docs)
print(f"Indexed {len(docs)} documents.")